In [1]:
!pip install keras

    100% |████████████████████████████████| 337kB 1.5MB/s ta 0:00:01


In [2]:
import datalab.bigquery as bq
import seaborn as sns
import pandas as pd
import numpy as np
import shutil
import os
import h5py
from sklearn import preprocessing
import keras
from pandas_gbq.gbq import GenericGBQException
import logging
import tensorflow as tf
print tf.__version__

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


1.5.0


In [3]:
PROJECT = 'go-de-internal'
BUCKET = 'go-de-internal-johanna'
REGION = 'europe-west1'
REPO = "/content/datalab/demo-cases/churn-prediction/neural-net"

In [4]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['REPO'] = REPO

In [5]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [6]:
%bash
PROJECT_ID=$PROJECT
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print response['serviceAccount']")

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

Authorizing the Cloud ML Service account service-462605511119@cloud-ml.google.com.iam.gserviceaccount.com to access files in go-de-internal-johanna


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   133    0   133    0     0    149      0 --:--:-- --:--:-- --:--:--   149
No changes to gs://go-de-internal-johanna/
No changes to gs://go-de-internal-johanna/churn-prediction/
No changes to gs://go-de-internal-johanna/churn-prediction/mv_ab_model_ads_build_sep.csv
No changes to gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-test.csv
No changes to gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-train.csv
No changes to gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-valid.csv
No changes to gs://go-de-internal-johanna/churn-prediction/neural_net/churn-telecommunication.csv
No changes to gs://go-de-internal-johanna/churn-prediction/neural_net/churn_trained/
No changes to gs://go-de-internal-johanna/churn-prediction/neural_net/churn_trained/churn_mo

In [7]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/churn-prediction/neural_net/
gsutil -m cp ${REPO}/*.csv gs://${BUCKET}/churn-prediction/neural_net/

go-de-internal-johanna


Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-test.csv#1520336666709749...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-train.csv#1520336666913302...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-valid.csv#1520336666710992...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-telecommunication.csv#1520336666418153...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn_trained/#1520336670979825...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn_trained/export/#1520336678399688...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn_trained/churn_model.hdf5#1520336677912198...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn_trained/export/saved_model.pb#1520336681936538...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn_trained/export/variables/#15203

In [8]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/churn-prediction/neural_net/
gsutil -m cp ${REPO}/*.csv gs://${BUCKET}/churn-prediction/neural_net/
OUTDIR=gs://${BUCKET}/churn-prediction/neural_net/churn_trained/
rm -rf vehicles.tar.gz churn_trained
gcloud ml-engine local train \
  --module-name=trainer.task \
  --package-path=${REPO}/einzelhandel/trainer \
  --job-dir $OUTDIR \
  -- \
  --train_files=${REPO}/churn-einzelhandel-train.csv \
  --eval_files=${REPO}/churn-einzelhandel-valid.csv  \
  --num_epochs=5 \
  --output_dir=$OUTDIR 

go-de-internal-johanna
Tensor("metrics/recall/div:0", shape=(), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'>
(<class 'keras.optimizers.RMSprop'>, {'epsilon': 1e-07, 'lr': 0.0010000000474974513, 'rho': 0.8999999761581421, 'decay': 0.0})
Epoch 1/5

100/100 [==============================] - 1s 10ms/step - loss: 0.5383 - acc: 0.7258 - precision: 0.7258 - recall: 0.7258
Epoch 2/5

100/100 [==============================] - 1s 8ms/step - loss: 0.4956 - acc: 0.7580 - precision: 0.7580 - recall: 0.7580
Epoch 3/5

100/100 [==============================] - 1s 8ms/step - loss: 0.4569 - acc: 0.7801 - precision: 0.7801 - recall: 0.7801
Epoch 4/5

100/100 [==============================] - 1s 8ms/step - loss: 0.4457 - acc: 0.7835 - precision: 0.7835 - recall: 0.7835
Epoch 5/5

100/100 [==============================] - 1s 7ms/step - loss: 0.4460 - acc: 0.7912 - precision: 0.7912 - recall: 0.7912


Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-test.csv#1520431147061741...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-train.csv#1520431147142429...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-valid.csv#1520431147059848...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-telecommunication.csv#1520431146519087...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/works-test.csv#1520431146519223...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/works-train.csv#1520431146884089...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/works-valid.csv#1520431146819256...
/ [7/7 objects] 100% Done                                                       
Operation completed over 7 objects.                                              
Copying file:///content/datalab/demo-cases/churn-prediction/neural-net/churn-einz

In [50]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/churn-prediction/neural_net
gsutil -m cp ${REPO}/*.csv gs://${BUCKET}/churn-prediction/neural_net/

go-de-internal-johanna


Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-test.csv#1520332655689918...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-train.csv#1520332655916552...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-valid.csv#1520332655689545...
Removing gs://go-de-internal-johanna/churn-prediction/neural_net/churn-telecommunication.csv#1520332655268372...
/ [4/4 objects] 100% Done                                                       
Operation completed over 4 objects.                                              
Copying file:///content/datalab/demo-cases/churn-prediction/neural-net/churn-einzelhandel-test.csv [Content-Type=text/csv]...
Copying file:///content/datalab/demo-cases/churn-prediction/neural-net/churn-einzelhandel-train.csv [Content-Type=text/csv]...
Copying file:///content/datalab/demo-cases/churn-prediction/neural-net/churn-einzelhandel-valid.csv [Content-Type=text/csv]...
Cop

In [51]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/churn-prediction/neural_net/
gsutil -m cp ${REPO}/*.csv gs://${BUCKET}/churn-prediction/neural_net/
OUTDIR=gs://${BUCKET}/churn-prediction/neural_net/churn_trained/
JOBNAME=churn_model_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=${REPO}/einzelhandel/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET/ \
  --scale-tier=BASIC \
  --runtime-version=1.2 \
  -- \
  --train_files="gs://${BUCKET}/churn_cloud/churn-einzelhandel-train.csv" \
  --eval_files="gs://${BUCKET}/churn_cloud/churn-einzelhandel-valid.csv"  \
  --num_epochs=100 \
  --output_dir=$OUTDIR \
  --train_steps=100 \
  --eval_steps=100 \
  --train_batch_size=40 \
  --eval_batch_size=40 \
  --learning_rate=0.003 \
  --first_layer_size=256 \
  --num_layers=5 \
  --scale_factor=0.25 \
  --eval_num_epochs=1 \
  --checkpoint_epochs=5

gs://go-de-internal-johanna/churn-prediction/neural_net/churn_trained europe-west1 churn_model_180306_103832
jobId: churn_model_180306_103832
state: QUEUED


CommandException: 1 files/objects could not be removed.
/tools/google-cloud-sdk/lib/googlecloudsdk/core/util/files.py:622: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  for chunk in iter(lambda: fp.read(4096), ''):
Job [churn_model_180306_103832] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe churn_model_180306_103832

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs churn_model_180306_103832


In [ ]:
!grep -A 2 add_argument trainer/task.py

In [ ]:
!grep -A 5 eval_metric trainer/model.py

In [ ]:
%writefile hyperparam.yaml
trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: val_ac
    maxTrials: 20
    maxParallelTrials: 1
    params:
    - parameterName: train_batch_size
      type: INTEGER
      minValue: 64
      maxValue: 512
      scaleType: UNIT_LOG_SCALE
    - parameterName: nbuckets
      type: INTEGER
      minValue: 10
      maxValue: 20
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: hidden_units
      type: CATEGORICAL
      categoricalValues: ["128 32", "256 128 16", "64 64 64 8"]       

In [ ]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/churn_cloud_hyper
gsutil -m cp ${REPO}/*.csv gs://${BUCKET}/churn_cloud_hyper/

In [ ]:
%bash
OUTDIR=gs://${BUCKET}/churn_cloud_hyper/churn_trained
JOBNAME=churn_model_hyper_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=${REPO}/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET/churn_cloud_hyper/ \
  --scale-tier=BASIC \
  --runtime-version=1.2 \
  --config=hyperparam.yaml \
  -- \
  --train_files="gs://${BUCKET}/churn_cloud_hyper/churn-einzelhandel-train.csv" \
  --eval_files="gs://${BUCKET}/churn_cloud_hyper/churn-einzelhandel-valid.csv"  \
  --num_epochs=100
  --output_dir=$OUTDIR \
  --train_steps=100 \
  --eval_steps=100 \
  --train_batch_size=40 \
  --eval_batch_size=40 \
  --learning_rate=0.003 \
  --first_layer_size=256 \
  --num_layers=5 \
  --scale_factor=0.25 \
  --eval_num_epochs=1 \
  --checkpoint_epochs=5